<a href="https://colab.research.google.com/github/laurarincon1/UDEA-ai4eng-20251/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

#Carga de datos

Se usa  el archivo kaggle.json para la carga de datos

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.' # Establece la variable de entorno 'KAGGLE_CONFIG_DIR' al directorio actual ('.').
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia #descarga los archivos asociados a esa competencia en el directorio actual.
!unzip udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip #Extrae todos los archivos contenidos en el .zip descargado del Kaggle.

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.13GB/s]
Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# Lee el archivo 'train.csv'
datos = pd.read_csv("train.csv")

In [ ]:
print(datos.shape)

(692500, 21)


#Limpia de datos

**Eliminación de variables no informativas**

In [ ]:
del(datos["ID"])
del(datos["ESTU_PRIVADO_LIBERTAD"])
del(datos["PERIODO"])
del(datos["FAMI_TIENEINTERNET.1"])


**Tratamiento de valores nulos**

In [ ]:
datos.isna().sum()


,0
ESTU_PRGM_ACADEMICO,0
ESTU_PRGM_DEPARTAMENTO,0
ESTU_VALORMATRICULAUNIVERSIDAD,6287
ESTU_HORASSEMANATRABAJA,30857
FAMI_ESTRATOVIVIENDA,32137
FAMI_TIENEINTERNET,26629
FAMI_EDUCACIONPADRE,23178
FAMI_TIENELAVADORA,39773
FAMI_TIENEAUTOMOVIL,43623
ESTU_PAGOMATRICULAPROPIO,6498


*Modificamos la variable ESTU_HORASSEMANATRABAJA para que sea una variable cuantitativa*

In [ ]:

# Diccionario de mapeo
mapeo_horas = {
    'Menos de 10 horas': 5,
    'Entre 11 y 20 horas': 15,
    'Entre 21 y 30 horas': 25,
    'Más de 30 horas': 35,
    '0':0
}

# Mapear los valores de texto a números
datos['ESTU_HORASSEMANATRABAJA'] = datos['ESTU_HORASSEMANATRABAJA'].map(mapeo_horas)


In [ ]:
#Remplazamos con la media
datos['ESTU_HORASSEMANATRABAJA']  = datos.ESTU_HORASSEMANATRABAJA.fillna(datos.ESTU_HORASSEMANATRABAJA.mean())

In [ ]:
datos['ESTU_HORASSEMANATRABAJA']=datos['ESTU_HORASSEMANATRABAJA'].astype(int)

*Modificamos la variable ESTU_VALORMATRICULAUNIVERSIDAD para que sea una variable cuantitativa*

In [ ]:
# Diccionario
mapeo_valor_matricula = {
    'Menos de 500 mil': 250000,
    'Entre 500 mil y menos de 1 millón': 750000,
    'Entre 1 millón y menos de 2.5 millones': 1750000,
    'Entre 2.5 millones y menos de 4 millones': 3250000,
    'Entre 4 millones y menos de 5.5 millones': 4750000,
    'Entre 5.5 millones y menos de 7 millones': 6250000,
    'Más de 7 millones': 8000000
}


# Mapear los valores de texto a números
datos['ESTU_VALORMATRICULAUNIVERSIDAD'] = datos['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapeo_valor_matricula)


In [ ]:
#Remplazamos con la media
datos['ESTU_VALORMATRICULAUNIVERSIDAD']  = datos.ESTU_VALORMATRICULAUNIVERSIDAD.fillna(datos.ESTU_VALORMATRICULAUNIVERSIDAD.mean())

In [ ]:
datos['ESTU_VALORMATRICULAUNIVERSIDAD']=datos['ESTU_VALORMATRICULAUNIVERSIDAD'].astype(int)

*Remplazo de datos categoricos faltantes(NA) por la Moda*

In [ ]:
modo = datos['FAMI_ESTRATOVIVIENDA'].mode()[0]
datos['FAMI_ESTRATOVIVIENDA'].fillna(modo, inplace=True)

<ipython-input-14-6521ba6ae174>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['FAMI_ESTRATOVIVIENDA'].fillna(modo, inplace=True)


In [ ]:
modo = datos['FAMI_TIENEAUTOMOVIL'].mode()[0]
datos['FAMI_TIENEAUTOMOVIL'].fillna(modo, inplace=True)

<ipython-input-15-2c285ab07e07>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos['FAMI_TIENEAUTOMOVIL'].fillna(modo, inplace=True)


In [ ]:
# Eliminar filas que contienen al menos un valor nulo
datos = datos.dropna()

In [ ]:
print(datos.shape)

(642269, 17)


***Uso de One Hot***

Convierte las columnas categóricas a codificación one-hot

In [ ]:
def replace_columns_with_onehot(d, col):
    def to_onehot(k):
      values = np.unique(k)  # Encuentra los valores únicos (las categorías posibles)
      r = np.r_[[np.argwhere(i==values)[0][0] for i in k]]  # Para cada valor en x, obtiene su índice en el array de valores únicos
      return np.eye(len(values))[r].astype(int)  # Devuelve la matriz identidad con filas seleccionadas por esos índices

    k = to_onehot(d[col].values)
    r = pd.DataFrame(k, columns=["%s_%d"%(col, i) for i in range(k.shape[1])], index=d.index).join(d)
    del(r[col])
    return r

In [ ]:
datos = replace_columns_with_onehot(datos, "FAMI_ESTRATOVIVIENDA")
datos = replace_columns_with_onehot(datos, "ESTU_PRGM_ACADEMICO")
datos = replace_columns_with_onehot(datos, "ESTU_PRGM_DEPARTAMENTO")
datos = replace_columns_with_onehot(datos, "FAMI_TIENEINTERNET")
datos = replace_columns_with_onehot(datos, "FAMI_EDUCACIONPADRE")
datos = replace_columns_with_onehot(datos, "FAMI_TIENELAVADORA")
datos = replace_columns_with_onehot(datos, "FAMI_TIENECOMPUTADOR")
datos = replace_columns_with_onehot(datos, "FAMI_EDUCACIONMADRE")
datos = replace_columns_with_onehot(datos, "FAMI_TIENEAUTOMOVIL")
datos = replace_columns_with_onehot(datos, "RENDIMIENTO_GLOBAL")
datos = replace_columns_with_onehot(datos, "ESTU_PAGOMATRICULAPROPIO")

In [ ]:
datos.head()

**Normalizar datos**

In [ ]:

scaler = MinMaxScaler()
cols_a_normalizar = [
    "ESTU_VALORMATRICULAUNIVERSIDAD",
    "ESTU_HORASSEMANATRABAJA",
    "coef_1", "coef_2", "coef_3", "coef_4"
]

datos[cols_a_normalizar] = scaler.fit_transform(datos[cols_a_normalizar])


In [ ]:
datos[cols_a_normalizar]

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,coef_1,coef_2,coef_3,coef_4
0,0.774194,0.142857,0.490107,0.427105,0.968750,0.804217
1,0.387097,0.000000,0.473364,0.441478,0.912500,0.795181
2,0.387097,1.000000,0.452055,0.439425,0.953125,0.795181
3,0.580645,0.000000,0.738204,0.353183,0.787500,0.572289
4,0.387097,0.714286,0.480974,0.476386,0.890625,0.885542
...,...,...,...,...,...,...
692495,0.064516,0.428571,0.360731,0.556468,0.846875,0.936747
692496,0.387097,1.000000,0.477930,0.492813,0.868750,0.783133
692497,0.193548,0.142857,0.435312,0.492813,0.981250,0.864458
692498,0.387097,0.142857,0.200913,0.874743,0.815625,0.987952


In [ ]:
datos.head()

,ESTU_PAGOMATRICULAPROPIO_0,ESTU_PAGOMATRICULAPROPIO_1,RENDIMIENTO_GLOBAL_0,RENDIMIENTO_GLOBAL_1,RENDIMIENTO_GLOBAL_2,RENDIMIENTO_GLOBAL_3,FAMI_TIENEAUTOMOVIL_0,FAMI_TIENEAUTOMOVIL_1,FAMI_EDUCACIONMADRE_0,FAMI_EDUCACIONMADRE_1,...,FAMI_ESTRATOVIVIENDA_3,FAMI_ESTRATOVIVIENDA_4,FAMI_ESTRATOVIVIENDA_5,FAMI_ESTRATOVIVIENDA_6,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,coef_1,coef_2,coef_3,coef_4
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0.774194,0.142857,0.490107,0.427105,0.968750,0.804217
1,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0.387097,0.000000,0.473364,0.441478,0.912500,0.795181
2,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0.387097,1.000000,0.452055,0.439425,0.953125,0.795181
3,1,0,1,0,0,0,1,0,0,0,...,1,0,0,0,0.580645,0.000000,0.738204,0.353183,0.787500,0.572289
4,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0.387097,0.714286,0.480974,0.476386,0.890625,0.885542
